In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import harpy as hp

#### Read in the data

In [ ]:
path = "/Users/arnedf/Downloads/xenium/"
#path = "/hive/hubmap/data/public/spatial-data-workshop/TMA2/xenium"

input_path = [
    os.path.join( path, "output-XETG00247__0021229__Region_1__20240319__210701"),
    os.path.join( path, "output-XETG00247__0021229__Region_3__20240319__210702"),
    os.path.join( path, "output-XETG00247__0021229__Region_4__20240319__210702"),
    #os.path.join( path, "output-XETG00247__0021229__Region_6__20240319__210702"),
    #os.path.join( path, "output-XETG00247__0021229__Region_7__20240319__210702"),
    #os.path.join( path, "output-XETG00247__0021229__Region_10__20240319__210702"),
    #os.path.join( path, "output-XETG00247__0021229__Region_11__20240319__210702"),
]

#regions = [ "region1", "region3", "region4", "region6", "region7", "region10", "region11" ]
regions = [ "region1", "region3", "region4", ]

OUTPUT_DIR = "/Users/arnedf/VIB/DATA/test_data/hubmap_training"
# OUTPUT_DIR = "/hive/user-workspaces/adefauw/1538/xenium_data"

crop=True

sdata = hp.io.xenium(
    input_path,
    to_coordinate_system=regions,
    aligned_images=True,
    cells_table=True,
    nucleus_labels=True,
    cells_labels=True,
    filter_gene_names=['Unassigned','NegControl','Deprecated'],
    output= os.path.join( OUTPUT_DIR, "sdata_xenium.zarr" ),
)

In [ ]:
sdata

In [ ]:
from napari_spatialdata import Interactive

Interactive(sdata )

Exercise:

Preprocess the data. Use the `harpy` functions `hp.im.min_max_filtering` and `hp.im.enhance_contrast`.


<details>
<summary>Click to reveal the solution</summary>

```python

from spatialdata.transformations import get_transformation

if crop is True:
  crd_region1 = [ 4000, 10000, 4000, 10000 ]
  crd_region3 = [ 5000, 12000, 0, 6000 ]
  crd_region4 = [ 400, 6400 , 0, 6000 ]
  crd_region6 = [ 600, 6600, 4000, 10000 ]
  crd_region7 = [ 2000, 8000, 9000, 15000 ]
  crd_region10 = [ 4000, 10000, 2600, 8600 ]
  crd_region11 = [ 2500, 8500, 3000, 9000 ]
  crd = [ crd_region1, crd_region3, crd_region4, crd_region6, crd_region7, crd_region10, crd_region11 ]
else:
    crd=None

for _crd,_region in zip(crd, regions, strict=False):

  sdata = hp.im.min_max_filtering(
      sdata,
      img_layer=f"morphology_focus_{_region}",
      output_layer=f"preprocessed_{_region}",
      size_min_max_filter=51,   # specify a single value to be used for all channels, or specify a list with a value for each channel
      crd = _crd,
      to_coordinate_system=_region,
      overwrite=True,
        )
  
  sdata = hp.im.enhance_contrast(
      sdata,
      img_layer=f"preprocessed_{_region}",
      output_layer=f"preprocessed_{_region}",
      contrast_clip=3.5,        # specify a single value to be used for all channels, or specify a list with a value for each channel
      chunks=None,
      overwrite=True,
        )
  
  # rechunk on disk, in this way we have 'optimal' chunk size for segmentation
  sdata=hp.im.add_image_layer(
      sdata,
      arr=sdata[ f"preprocessed_{_region}" ].data.rechunk( 2048 ),
      transformations=get_transformation( sdata[ f"preprocessed_{_region}" ], get_all=True ),
      output_layer = f"preprocessed_{_region}",
      overwrite=True,
      )

In [ ]:
from spatialdata.transformations import get_transformation

if crop is True:
  crd_region1 = [ 4000, 10000, 4000, 10000 ]
  crd_region3 = [ 5000, 12000, 0, 6000 ]
  crd_region4 = [ 400, 6400 , 0, 6000 ]
  crd_region6 = [ 600, 6600, 4000, 10000 ]
  crd_region7 = [ 2000, 8000, 9000, 15000 ]
  crd_region10 = [ 4000, 10000, 2600, 8600 ]
  crd_region11 = [ 2500, 8500, 3000, 9000 ]
  crd = [ crd_region1, crd_region3, crd_region4, crd_region6, crd_region7, crd_region10, crd_region11 ]
else:
    crd=None

for _crd,_region in zip(crd, regions, strict=False):

  sdata = hp.im.min_max_filtering(
      sdata,
      img_layer=f"morphology_focus_{_region}",
      output_layer=f"preprocessed_{_region}",
      size_min_max_filter=51,   # specify a single value to be used for all channels, or specify a list with a value for each channel
      crd = _crd,
      to_coordinate_system=_region,
      overwrite=True,
        )
  
  sdata = hp.im.enhance_contrast(
      sdata,
      img_layer=f"preprocessed_{_region}",
      output_layer=f"preprocessed_{_region}",
      contrast_clip=3.5,        # specify a single value to be used for all channels, or specify a list with a value for each channel
      chunks=None,
      overwrite=True,
        )
  
  # rechunk on disk, in this way we have 'optimal' chunk size for segmentation
  sdata=hp.im.add_image_layer(
      sdata,
      arr=sdata[ f"preprocessed_{_region}" ].data.rechunk( 2048 ),
      transformations=get_transformation( sdata[ f"preprocessed_{_region}" ], get_all=True ),
      output_layer = f"preprocessed_{_region}",
      overwrite=True,
      )

Exercise:

Visualize the data. Show both original and preprocessed images using the `harpy` function `hp.pl.plot_image`.


<details>
<summary>Click to reveal the solution</summary>

```python

hp.pl.plot_image(
    sdata,
    img_layer=[ "morphology_focus_region1", "preprocessed_region1", ],
    crd = crd_region1,
    to_coordinate_system="region1",
    figsize=(10,10),
      )

Segment using Cellpose.

In [ ]:
from dask.distributed import Client, LocalCluster

# # Create a local Dask cluster
cluster = LocalCluster(
     n_workers=8,              # Number of worker processes
     threads_per_worker=1,    # Number of threads per worker
     memory_limit="32GB",      # Memory limit per worker
 )

# # Connect a Client to the cluster
client = Client(cluster)

# # Print the Dask dashboard link
print(client.dashboard_link)

In [ ]:
from harpy.image import cellpose_callable

device = "cpu"  # mps broken in cellpose (macOS), see https://github.com/MouseLand/cellpose/issues/1063

# find good crop for each region

for _region in regions:

    sdata = hp.im.segment(
        sdata,
        img_layer=f"preprocessed_{_region}",  # or "morphology_focus_global"
        depth=200,
        model=cellpose_callable,
        # parameters that will be passed to the callable _cellpose
        pretrained_model = "nuclei",
        device = device,
        diameter=30,
        flow_threshold=0.9,
        cellprob_threshold=-4,
        channels=[1,0],
        output_labels_layer=f"segmentation_mask_{_region}",
        output_shapes_layer=f"segmentation_boundaries_{_region}",
        to_coordinate_system=_region,
        overwrite=True,
    )

client.close()

Exercise:

Convert the labels layers `cell_labels_region1` and `nucleus_labels_region1` to a shapes layer using the `harpy` function `harpy.sh.vectorize`.

<details>
<summary>Click to reveal the solution</summary>

```python

import dask
with dask.config.set(scheduler='processes'):
    sdata=hp.sh.vectorize( sdata, labels_layer="cell_labels_region1", output_layer="cell_shapes_region1", overwrite=True )
    sdata=hp.sh.vectorize( sdata, labels_layer="nucleus_labels_region1", output_layer="nucleus_shapes_region1", overwrite=True )

In [ ]:
hp.pl.plot_shapes(
    sdata,
    img_layer="preprocessed_region1",
    shapes_layer=["segmentation_boundaries_region1", "cell_shapes_region1"],
    crd = [4500, 5500, 4500, 5500],
    to_coordinate_system="region1",
    figsize=(8,8),
      )

In [ ]:
hp.pl.plot_shapes(
    sdata,
    img_layer="preprocessed_region1",
    shapes_layer=["segmentation_boundaries_region1", "cell_shapes_region1"],
    crd = [8500, 10000, 8500, 10000],
    to_coordinate_system="region1",
    figsize=(8,8),
      )

We create one AnnData object, with data from all regions.

In [ ]:
for _region in regions:

    sdata=hp.tb.allocate( 
        sdata,
        labels_layer=f"segmentation_mask_{_region}",
        points_layer=f"transcripts_{_region}",
        output_layer="table_harpy",
        to_coordinate_system=_region,
        chunks=None,
        append=True,
        overwrite=True,
        update_shapes_layers=False,
    )

In [ ]:
sdata[ "table_harpy" ].obs[ "fov_labels" ].cat.categories.tolist()

In [ ]:
sdata[ "table_harpy" ].obs.head()

Preprocessing:

In [ ]:
sdata = hp.tb.preprocess_transcriptomics(
    sdata,
    labels_layer=sdata[ "table_harpy" ].obs[ "fov_labels" ].cat.categories,
    table_layer="table_harpy",
    output_layer="table_harpy",  # write results to a new slot, we could also write to the same slot (when passing overwrite==True).
    min_counts=10,
    min_cells=5,
    size_norm=True,
    n_comps=50,
    overwrite=True,
    update_shapes_layers=False,
)

sdata = hp.tb.filter_on_size(
    sdata,
    labels_layer=sdata[ "table_harpy" ].obs[ "fov_labels" ].cat.categories,
    table_layer="table_harpy",
    output_layer="table_harpy",
    min_size=500,
    max_size=100000,
    update_shapes_layers=False,
    overwrite=True,
)

Leiden clustering:

In [ ]:
import scanpy as sc

sdata = hp.tb.leiden(
    sdata,
    labels_layer=sdata[ "table_harpy" ].obs[ "fov_labels" ].cat.categories,
    table_layer="table_harpy",
    output_layer="table_harpy",
    calculate_umap=True,
    calculate_neighbors=True,
    n_pcs=17,
    n_neighbors=35,
    resolution=0.8,
    rank_genes=True,
    key_added="leiden",
    overwrite=True,
)

sc.pl.umap(
    sdata.tables["table_harpy"],
    color=["leiden"],
    show=True,
    )

sc.pl.rank_genes_groups(
    sdata.tables["table_harpy"],
    n_genes=8,
    sharey=False,
    show=True,
    )

In [ ]:
sc.pl.umap(
    sdata.tables["table_harpy"],
    color=["fov_labels"],
    show=True,
    )

Exercise:

Plot the leiden clusters, either using `harpy.pl.plot_shapes` or `spatialdata-plot`.

<details>
<summary>Click to reveal the solution</summary>

```python

hp.pl.plot_shapes(
    sdata,
    img_layer="preprocessed_region1",
    shapes_layer="segmentation_boundaries_region1",
    region = "segmentation_mask_region1",
    to_coordinate_system="region1",
    table_layer="table_harpy",
    column="leiden",
    alpha=1,
    linewidth=0,
    channel="DAPI",
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
ax = plt.gca()

column = "leiden"

adata = sdata.tables[ "table_harpy" ]

sdata_small = sdata.query.bounding_box(
    min_coordinate=[4000, 4000], max_coordinate=[10000, 10000], axes=("x", "y"), target_coordinate_system="region1"
)

sdata_small.pl.render_labels("segmentation_mask_region1", color=column, cmap=None, method="datashader", fill_alpha=1, table_name= "table_harpy").pl.show(
    coordinate_systems="region1", ax=ax
)

In [ ]:
from napari_spatialdata import Interactive

# del sdata[ "table_harpy"].uns[ "leiden_colors" ]

Interactive( sdata )